In [1]:
import pandas as pd
from numpy import int64
from datetime import date
from datetime import datetime

In [2]:
# PARÁMETROS
CARPETA_NIVS = 'JAVA' # Se cambia de acuerdo al perfil de informe a generar
NOMBRE_COLUMNA_CALIFICACIONES = 'Calificación/5.00'
EST_MATRICULADOS = pd.read_excel(f'./INPUT/{CARPETA_NIVS}/EstudiantesMatriculadosNiv.xlsx', engine = 'openpyxl')
VERSIONES_NIV = [f'./INPUT/{CARPETA_NIVS}/2022-2-2202-NJ1-Presencial-Examen de nivelación-calificaciones.csv']
#VERSIONES_NIV = [f'./INPUT/{CARPETA_NIVS}/2022-2-2202-AJP-Presencial-Examen de nivelación-calificaciones.csv']

UMBRAL_APROBACION = 3
ARRASTRE = True
UMBRAL_ARRASTRE = 3.7

COLUMNAS_EVAL = True
NUMERO_EVAL = 5
NOMBRE_EVAL = 'RETO'

GENERAR_ARCHIVOS_CSV = True # Requiere que NUMERO_EVAL esté configurado correctamente
ANIO = 2022
# PERIODO = [1, 2] # Los periodos deben coincidir 1:1 con el código de la asignatura
# CODIGO_ASIGNATURA = [2411, 2511]
PERIODO = [2] # Los periodos deben coincidir 1:1 con el código de la asignatura
CODIGO_ASIGNATURA = [2511]
GRUPO = 'NIV'
CSV_DIFERENCIAL = False # Si se requiere que los archivos CSV generados NO compartan datos de cargue de un punto anterior
if CSV_DIFERENCIAL:
    # Se debe de cargar el formato de nivelación que se usó como punto de control
    # Eventualmente se podría convertir en una lista
    CHECKPOINT = pd.read_excel(f'./INPUT/{CARPETA_NIVS}/INFORME_NIVELACIÓN_2022-07-19_Hora-11-VERF-21-12PM.xlsx', sheet_name='Estudiantes que aprobaron', engine = 'openpyxl')

In [3]:
# PROCESAMIENTO
CALIFICACIONES = []
for cal in VERSIONES_NIV:
    CALIFICACIONES.append(pd.read_csv(cal, sep = ',')[:-1])
CALIFICACIONES = pd.concat(CALIFICACIONES, ignore_index = True) if len(CALIFICACIONES) > 1 else CALIFICACIONES[0]

if ARRASTRE:
    print ("== Arrastre encendido ==")
    for index, row in CALIFICACIONES.iterrows():
        if UMBRAL_APROBACION <= row[NOMBRE_COLUMNA_CALIFICACIONES] < UMBRAL_ARRASTRE:
            print ("Modificado: ", row['Nombre de usuario'], row[NOMBRE_COLUMNA_CALIFICACIONES])
            CALIFICACIONES[NOMBRE_COLUMNA_CALIFICACIONES].loc[index] = UMBRAL_ARRASTRE
            
aprobados = CALIFICACIONES[CALIFICACIONES[NOMBRE_COLUMNA_CALIFICACIONES] >= UMBRAL_APROBACION]
cedulas = []
for index, row in aprobados.iterrows():
    cedulas.append(EST_MATRICULADOS[EST_MATRICULADOS['Nombre de usuario'] == row['Nombre de usuario']]['Cédula'].iloc[0])
aprobados['Cédula'] = cedulas

if COLUMNAS_EVAL:
    nombresCols = []
    for i in range(NUMERO_EVAL):
        nombreCol = NOMBRE_EVAL + '_' + str(i+1)
        aprobados[nombreCol] = aprobados[NOMBRE_COLUMNA_CALIFICACIONES]
        nombresCols.append(nombreCol)

== Arrastre encendido ==
Modificado:  2221471.0 3.12
Modificado:  2220864.0 3.12


C:\Users\radou\AppData\Local\Temp/ipykernel_9696/2118158090.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CALIFICACIONES[NOMBRE_COLUMNA_CALIFICACIONES].loc[index] = UMBRAL_ARRASTRE


Modificado:  2222451.0 3.12
Modificado:  2220076.0 3.12
Modificado:  2222185.0 3.12
Modificado:  2224049.0 3.12
Modificado:  2220787.0 3.12
Modificado:  2220753.0 3.12
Modificado:  2224564.0 3.12
Modificado:  2220528.0 3.12
Modificado:  2222540.0 3.12
Modificado:  2224504.0 3.12
Modificado:  2222168.0 3.12
Modificado:  2221589.0 3.12
Modificado:  2224418.0 3.12
Modificado:  2220763.0 3.12
Modificado:  2221692.0 3.12


C:\Users\radou\AppData\Local\Temp/ipykernel_9696/2118158090.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aprobados['Cédula'] = cedulas
C:\Users\radou\AppData\Local\Temp/ipykernel_9696/2118158090.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aprobados[nombreCol] = aprobados[NOMBRE_COLUMNA_CALIFICACIONES]
C:\Users\radou\AppData\Local\Temp/ipykernel_9696/2118158090.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [4]:
porNivelar = pd.merge(EST_MATRICULADOS, aprobados, on=['Nombre de usuario'], how="outer", indicator=True
              ).query('_merge=="left_only"')
porNivelar = porNivelar[['Nombre_x', 'Apellido(s)_x', 'Nombre de usuario','Cédula_x','Número de ID_x']]

In [5]:
porNivelar.rename(columns = {
    'Nombre_x' : 'Nombres',
    'Apellido(s)_x' : 'Apellidos',
    'Nombre de usuario' : 'Código UIS',
    'Cédula_x' : 'Cédula',
    'Número de ID_x' : 'ID Moodle'
}, inplace = True)

In [6]:
columnasBuenas = ['Nombre', 'Apellido(s)', 'Nombre de usuario', 'Cédula', 'Número de ID', 'Comenzado el', 'Finalizado', 'Tiempo requerido']
if COLUMNAS_EVAL:
    for col in nombresCols:
        columnasBuenas.append(col)
    columnasBuenas += [NOMBRE_COLUMNA_CALIFICACIONES]

aprobados = aprobados[columnasBuenas]
aprobados.rename(columns = {
    'Nombre' : 'Nombres',
    'Apellido(s)': 'Apellidos',
    'Nombre de usuario' : 'Código UIS',
    'Número de ID': 'ID Moodle',
}, inplace = True)
aprobados['Código UIS'] = aprobados['Código UIS'].astype(int64, errors='ignore')

In [7]:
nombreArchivo = f'INFORME_NIVELACIÓN_{CARPETA_NIVS}_' + str(date.today()) + '_Hora-' + str(datetime.now().hour) + '.xlsx'
writer = pd.ExcelWriter('./OUTPUT/' + nombreArchivo, engine='openpyxl')
porNivelar.to_excel(writer, sheet_name = 'Estudiantes pendientes nivelar', index = False)    
aprobados.to_excel(writer, sheet_name = 'Estudiantes que aprobaron', index = False)  
writer.close()

In [8]:
if GENERAR_ARCHIVOS_CSV:
    conjuntoCSV = aprobados.copy()
    if CSV_DIFERENCIAL:
        conjuntoCSV = conjuntoCSV.merge(CHECKPOINT, on='Código UIS', how='left', indicator = True, suffixes=('', '_y')).query('_merge == "left_only"').drop('_merge', 1)
        conjuntoCSV.drop(conjuntoCSV.filter(regex='_y$').columns, axis=1, inplace = True)
    for j in range(len(CODIGO_ASIGNATURA)):        
        csvParcial = {
            'CODIGO ESTUDIANTE': [],
            'AÑO': [],
            'PERIODO': [],
            'CODIGO ASIGNATURA': [],
            'NUMERO PARCIAL': [],
            'GRUPO': [],
            'NOTA': []
        }
        csvDef = {
            'CODIGO ESTUDIANTE': [],
            'AÑO': [],
            'PERIODO': [],
            'CODIGO ASIGNATURA': [],
            'GRUPO': [],
            'NOTA DEFINITIVA': []
        }
        periodo = PERIODO[j]
        codigoAsig = CODIGO_ASIGNATURA[j]
        for index, row in conjuntoCSV.iterrows():
            csvDef['CODIGO ESTUDIANTE'].append(row['Código UIS'])
            csvDef['AÑO'].append(ANIO)
            csvDef['PERIODO'].append(periodo)
            csvDef['CODIGO ASIGNATURA'].append(codigoAsig)
            csvDef['GRUPO'].append(GRUPO)
            csvDef['NOTA DEFINITIVA'].append(row[NOMBRE_COLUMNA_CALIFICACIONES])
            for i in range (1, NUMERO_EVAL + 1):
                csvParcial['CODIGO ESTUDIANTE'].append(row['Código UIS'])
                csvParcial['AÑO'].append(ANIO)
                csvParcial['PERIODO'].append(periodo)
                csvParcial['CODIGO ASIGNATURA'].append(codigoAsig)
                csvParcial['NUMERO PARCIAL'].append(i)
                csvParcial['GRUPO'].append(GRUPO)
                csvParcial['NOTA'].append(row[NOMBRE_COLUMNA_CALIFICACIONES])
        pd.DataFrame(csvDef).to_csv(f'./OUTPUT/CSVS/CAMBIO_NOTAS_DEFINITIVAS_{CARPETA_NIVS}_{codigoAsig}_' + str(date.today()) + '.csv', sep=';', encoding = 'utf-8', index=False)
        pd.DataFrame(csvParcial).to_csv(f'./OUTPUT/CSVS/CAMBIO_NOTAS_PARCIALES_{CARPETA_NIVS}_{codigoAsig}_' + str(date.today()) + '.csv', sep=';', encoding = 'utf-8', index=False)